In [ ]:
import glob

import pandas as pd
from prophet import Prophet

In [ ]:
df = pd.concat(map(pd.read_csv, glob.glob('The Crown & Cutlery/*.csv')), ignore_index=True)
df.fillna(0)

regressors = ('Temperature','Humidity', 'Is_Raining', 'Is_Sunny', 'Is_Holiday')

In [ ]:
df["y"] = df[['Dessert_Waste_kg', 'Soup_Waste_kg',
       'Main Course_Waste_kg', 'Appetizer_Waste_kg', 'Salad_Waste_kg',
       'Beverage_Waste_kg']].sum(axis=1)
df.rename(columns={"Date": "ds"}, inplace=True)

In [ ]:
m = Prophet()
m.add_country_holidays(country_name='UK')
for i in regressors:
    m.add_regressor(i)
m.fit(df)

In [ ]:
future = m.make_future_dataframe(periods=30)

for i in regressors:
    dfr = df[['ds', i]]
    dfr = dfr.rename(columns={i: "y"})
    rm = Prophet()
    rm.fit(dfr)
    rfuture = rm.make_future_dataframe(periods=30)
    rforecast = rm.predict(rfuture)
    future[i] = rforecast["yhat"]

In [ ]:
forecast = m.predict(future)

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)